In [1]:
# Import packages
import os
import warnings

#import matplotlib.pyplot as plt
import numpy.ma as ma
import xarray as xr
import rioxarray as rxr
#from shapely.geometry import mapping, box
#import geopandas as gpd
#import earthpy as et
#import earthpy.plot as ep
import numpy as np
from glob import glob

warnings.simplefilter('ignore')


ImportError: libproj.so.22: cannot open shared object file: No such file or directory

In [ ]:
def combine_tifs(tif_list):
    """A function that combines a list of tifs in the same CRS
    and of the same extent into an xarray object

    Parameters
    ----------
    tif_list : list
        A list of paths to the tif files that you wish to combine.

    Returns
    -------
    An xarray object with all of the tif files in the listmerged into 
    a single object.

    """

    out_xr = []
    for i, tif_path in enumerate(tif_list):
        out_xr.append(rxr.open_rasterio(tif_path, masked=True).squeeze())
        out_xr[i]["band"] = i+1

    return xr.concat(out_xr, dim="band")

In [102]:
data_path = os.path.join('/home/omar/Data/MODIS/Pecs/')
file_name = 'MOD06_L2.A2020231.1125.061.2020231194635.hdf'

In [103]:
# Open just the desired_attributes that you want to process and masking the no-data values
desired_attributes = ['cloud_top_height_1km', 'cloud_top_pressure_1km', 'cloud_top_temperature_1km']
#'Cloud_Effective_Emissivity', 'Cloud_Effective_Radius', 'Cloud_Phase_Optical_Properties'

In [104]:
# Notice that here, you get a single xarray object with just the desired_attributes that
# you want to work with
modis_dataset = rxr.open_rasterio(os.path.join(data_path,file_name),
                                    masked=True,
                                    variable=desired_attributes).squeeze()


In [105]:
modis_dataset.to_array()

<xarray.DataArray (variable: 3, y: 2030, x: 1354)>
array([[[ 9300.,  9300.,  8000., ...,  9050.,  8800.,  7550.],
        [ 9300.,  8800.,  8000., ...,  9300.,  8800.,  7550.],
        [ 9300.,  8250.,  8000., ...,  9850.,  9600.,  8800.],
        ...,
        [ 7300.,  7300.,  7300., ...,  7300.,  6850.,  2350.],
        [ 7300.,  7300.,  7300., ...,  7050.,  7300.,  6600.],
        [ 7300.,  7300.,  7300., ...,  2250.,  6850.,  5550.]],

       [[  700.,   700.,  1950., ...,   950.,  1200.,  2450.],
        [  700.,  1200.,  1950., ...,   700.,  1200.,  2450.],
        [  700.,  1700.,  1950., ...,   200.,   450.,  1200.],
        ...,
        [ 2600.,  2600.,  2650., ...,  2800.,  3300., 11150.],
        [ 2600.,  2600.,  2650., ...,  3050.,  2800.,  3600.],
        [ 2600.,  2600.,  2650., ..., 11500.,  3300.,  5000.]],

       [[13151., 13151., 12763., ..., 13461., 13284., 12466.],
        [13151., 13109., 12763., ..., 13626., 13284., 12466.],
        [13151., 12910., 12763., ..., 13785., 13626., 13105.],
        ...,
        [12400., 12400., 12396., ..., 12864., 12560.,  7352.],
        [12400., 12400., 12396., ..., 12703., 12864., 12422.],
        [12400., 12400., 12396., ...,  7044., 12560., 11842.]]])
Coordinates:
  * y            (y) float64 0.5 1.5 2.5 3.5 ... 2.028e+03 2.028e+03 2.03e+03
  * x            (x) float64 0.5 1.5 2.5 3.5 ... 1.352e+03 1.352e+03 1.354e+03
    band         int64 1
    spatial_ref  int64 0
  * variable     (variable) <U25 'cloud_top_pressure_1km' ... 'cloud_top_temp...
Attributes: (12/77)
    ALGORITHMPACKAGEACCEPTANCEDATE:           June 1997
    ALGORITHMPACKAGEMATURITYCODE:             at-launch
    ALGORITHMPACKAGENAME:                     ATBD-MOD-04 and ATBD-MOD-05
    ALGORITHMPACKAGEVERSION:                  2
    ALGORITHM_VERSION_CLOUD_PHASE_IR:         1
    ALGORITHM_VERSION_CLOUD_PROPERTY_VIS:     1
    ...                                       ...
    ThinCloudDetectedPct_IR:                  14.19
    title:                                    MODIS Level 2 Cloud Properties ...
    VERSIONID:                                61
    WaterCloudDetectedPct_IR:                 66.28
    Water_Phase_Forced:                       n
    WESTBOUNDINGCOORDINATE:                   -21.6009860094125

In [67]:
output_tif_path = os.path.join(data_path, "tifs", "mod06", "mod06")

In [68]:
modis_dataset['cloud_top_height_1km'].astype(np.int16).rio.to_raster(output_tif_path + '_CTH.tif')

In [69]:
modis_dataset['cloud_top_pressure_1km'].astype(np.int16).rio.to_raster(output_tif_path + '_CTP.tif')

In [70]:
modis_dataset['cloud_top_temperature_1km'].astype(np.int16).rio.to_raster(output_tif_path + '_CTT.tif')

In [79]:
# #Load them again    
# modis_dataset_list = glob(os.path.join(data_path, "tifs","mod06","*.tif"))

# # Sort the list of bands
# modis_dataset_list.sort()

In [80]:
# # Open file list with function
# modis_dataset = combine_tifs(modis_dataset_list)

In [95]:
modis_dataset.to_array


<xarray.DataArray (y: 2030, x: 1354)>
array([[  700.,   700.,  1950., ...,   950.,  1200.,  2450.],
       [  700.,  1200.,  1950., ...,   700.,  1200.,  2450.],
       [  700.,  1700.,  1950., ...,   200.,   450.,  1200.],
       ...,
       [ 2600.,  2600.,  2650., ...,  2800.,  3300., 11150.],
       [ 2600.,  2600.,  2650., ...,  3050.,  2800.,  3600.],
       [ 2600.,  2600.,  2650., ..., 11500.,  3300.,  5000.]])
Coordinates:
  * y            (y) float64 0.5 1.5 2.5 3.5 ... 2.028e+03 2.028e+03 2.03e+03
  * x            (x) float64 0.5 1.5 2.5 3.5 ... 1.352e+03 1.352e+03 1.354e+03
    band         int64 1
    spatial_ref  int64 0
    variable     <U25 'cloud_top_height_1km'
Attributes: (12/77)
    ALGORITHMPACKAGEACCEPTANCEDATE:           June 1997
    ALGORITHMPACKAGEMATURITYCODE:             at-launch
    ALGORITHMPACKAGENAME:                     ATBD-MOD-04 and ATBD-MOD-05
    ALGORITHMPACKAGEVERSION:                  2
    ALGORITHM_VERSION_CLOUD_PHASE_IR:         1
    ALGORITHM_VERSION_CLOUD_PROPERTY_VIS:     1
    ...                                       ...
    ThinCloudDetectedPct_IR:                  14.19
    title:                                    MODIS Level 2 Cloud Properties ...
    VERSIONID:                                61
    WaterCloudDetectedPct_IR:                 66.28
    Water_Phase_Forced:                       n
    WESTBOUNDINGCOORDINATE:                   -21.6009860094125